In [4]:
import numpy as np
import pandas as pd

# Generation of initial population of random antibodies

In [2]:
# m: number of machines
# p: number of parts
# population_size : number of antibody of the initial population
def generation_initial_population(p, m, population_size):
    MaxCell = min(p,m) #calculation of max number of cells
    number_of_zeros = MaxCell - 1 #number of zeros in each antibody
    antibodies = np.empty(shape=(population_size, p+m+number_of_zeros), dtype=int)
    antibody = np.append(np.array([*range(1,p+m+1)]), np.zeros(number_of_zeros,dtype=int))
    for i in range(0,population_size):
        np.random.shuffle(antibody) #random positions in the array
        antibodies[i] = antibody
    return antibodies

In [3]:
generation_initial_population(p = 7, m = 5, population_size = 3)

array([[12,  6,  1,  3, 10,  0,  7,  2, 11,  0,  5,  0,  0,  8,  4,  9],
       [ 4,  3,  0,  0,  5,  8,  0,  1,  0, 10,  9, 12,  2,  7,  6, 11],
       [ 5,  0,  0,  3,  6,  8, 11, 12,  0,  4,  9,  1, 10,  0,  7,  2]])

# Evaluate all existing antibodies and compute their affinities

Lectura de los datos del problema

In [222]:
f = open('data/instances/testset_a/5x7_Waghodekar_Sahu(1984)[Problem-2].txt', 'r')
lines = [line.split('\n') for line in f]
f.close()

m,p = [int(num) for num in lines[8][0].split(' ')] #m: number of machines, p: number of parts

machines=[[m,p]]
for i in range(9,9+m):
    #machines.append([int(num) for num in lines[i][0].split(' ')])
    machines.append([int(lines[i][0].split(' ')[j]) for j in range(1,m)])

columns, rows = ['M'+str(i) for i in range(1,m+1)], ['P'+str(i) for i in range(1,p+1)]
m_p_matrix = pd.DataFrame(columns= columns, index= rows)

ones_zeros = []
for i in range(1,len(machines)):
    aux = []
    for j in range(1,p+1):
        if j in machines[i]: aux.append(1)
        else: aux.append(0)
    ones_zeros.append(aux)

for i in range(0,len(columns)):
    m_p_matrix[columns[i]] = ones_zeros[i]

m_p_matrix

,M1,M2,M3,M4,M5
P1,1,0,0,1,0
P2,0,1,0,1,1
P3,0,1,1,1,0
P4,0,1,1,1,1
P5,1,1,1,0,1
P6,1,0,1,0,1
P7,1,0,0,0,0


Traducción del formato del dataset a matriz trabajo-estación

,M1,M2,M3,M4,M5
P1,1,0,0,1,0
P2,0,1,0,1,1
P3,0,1,1,1,0
P4,0,1,1,1,1
P5,1,1,1,0,1
P6,1,0,1,0,1
P7,1,0,0,0,0


# Select N% of antibodies with highest affinities

# Clone the selected antibodies